# Laboratorio 4
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Data Science - Sección 10

In [2]:
# Importar librerías
import pandas as pd

#### 2. Cargar el archivo de datos.

In [3]:
# 1. Ya se descargo el archivo train.csv
data_frame = pd.read_csv('./data/train.csv')
data_frame

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


#### 3. Limpie y preprocese los datos (Describir de forma detallada las actividades de preprocesamiento que se llevó a cabo)

In [4]:
# Convertir el texto en minúsculas


#### 4. Obtener frecuencia de las palabras. (Tanto de los tweets de desastres como de los que no).

##### ¿Qué palabras cree que le servirán para hacer un mejor modelo de clasificación?

...

##### ¿Vale la pena explorar bigramas para analizar contexto?

...

#### 5. Realizar un análisis exploratorio de los datos para entrenderlos mejor, documente todos los análisis.

In [5]:
# type here

#### 6. Elabore una función en el que el usuario ingrese un tweet y el sistema lo clasifica en desastre o no.

In [6]:
# type here